# Assignment: Подробнее о методах кластеризации

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

Нас будет интересовать файл `checkins.dat`. Открыв его, увидим следующую структуру:

```
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...
```

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

```
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...
```

С помощью `pandas` построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [1]:
import csv

with open('checkins.dat', 'r') as fin:
    with open('checkins.csv', 'w') as fout:
        for line in fin:
            newline = list(map(str.strip, line.split('|')))
            if len(newline) == 6 and newline[3] and newline[4]:
                csv.writer(fout).writerow(newline)

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [3]:
data = pd.read_csv('checkins.csv')

drop_features = ['id','user_id', 'venue_id', 'created_at']
data.drop(drop_features, axis=1, inplace=True)

print(data.shape)
data.head()

(396634, 2)


,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396632 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [4]:
from sklearn.cluster import MeanShift

data_reduced = data[:10**5]

ms_model = MeanShift(bandwidth=0.1)
ms_model.fit(data_reduced)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [21]:
significant_centers = []

for i in range(len(ms_model.cluster_centers_)):
    if ms_model.labels_[i] > 15:
        coordinates = ms_model.cluster_centers_[i]
        significant_centers.append(coordinates)
        
len(significant_centers)

1737

Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт `mapcustomizer.com` имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

```
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...
```

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико, чтобы не повлиять на кластеризацию или сортировку.

In [23]:
import numpy as np

In [24]:
offices = np.array([[33.751277, -118.188740],
                    [25.867736, -80.324116],
                    [51.503016, -0.075479],
                    [52.378894, 4.885084], 
                    [39.366487, 117.036146], 
                    [-33.868457, 151.205134]]) 

In [27]:
def distance(office, center):
    return (office[0] - center[0])**2 + (office[1] - center[1])**2

In [32]:
dist_location = {}

for center in significant_centers:
    dist = np.inf
    
    for office in offices:
        d = distance(office, center)
        if d < dist: dist = d
        
    dist_location[dist] = center

In [42]:
closest_centers = []

for index, distance in enumerate(sorted(dist_location)):
    if index == 20: break
    closest_centers.append(dist_location[distance])
    
closest_centers

[array([-33.86063043, 151.20477593]),
 array([52.37296399,  4.89231722]),
 array([51.50299126, -0.12553729]),
 array([ 25.78581242, -80.21793804]),
 array([-34.00190615, 151.12806905]),
 array([ 26.01009825, -80.19999059]),
 array([-33.9522629, 151.0321372]),
 array([  33.88832534, -118.04892817]),
 array([52.388501  ,  4.63376547]),
 array([51.50647877, -0.36517727]),
 array([52.2738177,  5.1643425]),
 array([ 26.12086266, -80.15890668]),
 array([ 39.121079, 117.214389]),
 array([  33.81730643, -117.89124917]),
 array([  33.67430266, -117.85878927]),
 array([51.2329101, -0.3297445]),
 array([52.080058  ,  5.10513275]),
 array([  34.13146015, -118.11801181]),
 array([51.1133388, -0.1829137]),
 array([  34.12740222, -118.35188373])]

In [51]:
sorted_centers = [[] for i in range(20)]

for center1 in closest_centers:
    cnt = 0
    
    for center2 in closest_centers:
        if center1[0] > center2[0]: cnt += 1
            
    sorted_centers[cnt] = center1

In [52]:
sorted_centers

[array([-34.00190615, 151.12806905]),
 array([-33.9522629, 151.0321372]),
 array([-33.86063043, 151.20477593]),
 array([ 25.78581242, -80.21793804]),
 array([ 26.01009825, -80.19999059]),
 array([ 26.12086266, -80.15890668]),
 array([  33.67430266, -117.85878927]),
 array([  33.81730643, -117.89124917]),
 array([  33.88832534, -118.04892817]),
 array([  34.12740222, -118.35188373]),
 array([  34.13146015, -118.11801181]),
 array([ 39.121079, 117.214389]),
 array([51.1133388, -0.1829137]),
 array([51.2329101, -0.3297445]),
 array([51.50299126, -0.12553729]),
 array([51.50647877, -0.36517727]),
 array([52.080058  ,  5.10513275]),
 array([52.2738177,  5.1643425]),
 array([52.37296399,  4.89231722]),
 array([52.388501  ,  4.63376547])]

Предположим, вы получили итоговые 20 точек в следующем виде:

```
latitude, longitude

2.2,-2.2

3.3,-3.3

1.1,-0.1

1.1,-1.1

…

19.19,-19.19
```

Отсортируйте полученный список пар (широта, долгота) по возрастанию (пары сравниваются сначала по первому элементу, затем по второму):

```
latitude, longitude

1.1,-1.1

1.1,-0.1

2.2,-2.2

3.3,-3.3

…

19.19,-19.19
```

В итоговый файл с ответами выведите по очереди широту и долготу каждой точки (в том же порядке), получив одну строчку из 40 чисел, разделенных пробелами:

`1.1 -1.1 1.1 -0.1 2.2 -2.2 3.3 -3.3 … 19.19 -19.19`

In [67]:
with open('answer.txt', 'w') as fout:
    fout.write(f"{sorted_centers[0][0]} {sorted_centers[0][1]}")